See API documentation for full set of search parameters:

https://api-staging.terramatch.org/research-service/documentation/api#/default/ 

In [12]:
import requests
import yaml
import json
import pandas as pd

In [13]:
# Set up token access
auth_path = '../secrets.yaml'
with open(auth_path) as auth_file:
    auth = yaml.safe_load(auth_file)
headers = {
    'Authorization': f"Bearer {auth['access_token']}"
    }

In [14]:
# API URLs
staging_url = "https://api-staging.terramatch.org/research/v3/sitePolygons?" # use for testing queries
prod_url = "https://api.terramatch.org/research/v3/sitePolygons?" # Use to pull data for analysis

In [15]:
# Define search parameters
params = {'projectId[]': '389aad5b-6577-4cea-bf9f-446dcfd94966',
    'polygonStatus[]':'approved',
          'includeTestProjects': 'false',
          'page[size]': '100'
          }

In [5]:
def pull_tm_api_data(url, headers, params):
    results = []
    last_record = ''
    while True:
    # send request
        response = requests.get(url, headers=headers,params=params)
        # check status code
        if response.status_code != 200:
            raise ValueError('Request failed with status code ' + str(response.status_code))
        # parse response data
        for idx in range(0, response.json()['meta']['page']['total']):
            data = response.json()['data'][idx]['attributes']
            data['poly_id'] =response.json()['data'][idx]['meta']['page']['cursor']
        # add results to list
            results.append(data)
            if idx == ((response.json()['meta']['page']['total']) - 1):
                new_last_record = response.json()['data'][idx]['meta']['page']['cursor']
        # check if there are more pages
        if (last_record != new_last_record):
            last_record = new_last_record
            params['page[after]'] =last_record
        else:
            break
    return results

In [6]:
project_results = pull_tm_api_data(staging_url, headers, params)

In [ ]:
project_df = pd.DataFrame(project_results)
project_df